# Kết nối Elasticsearch

In [7]:
import pandas as pd
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from indexMapping import indexMapping


es = Elasticsearch(
"http://localhost:9200",
)
client = es.info()
print(client)

{'name': '35f9c357c43a', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'mdVvttxWQ4OwubE3lDGk6A', 'version': {'number': '8.15.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179', 'build_date': '2024-08-05T10:05:34.233336849Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


# Lọc CSV và lưu vào cleaned_data.csv

In [8]:
df = pd.read_csv("../info/info.csv")
df.isna()

,Price,Area,Position,Type,Benefit
0,False,False,False,False,False
1,False,False,False,False,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
74300,True,True,True,False,False
74301,True,True,True,False,False
74302,False,False,False,False,False
74303,False,False,False,False,False


In [9]:


df = df.drop_duplicates()
df = df.dropna()
df.to_csv("../info/cleaned_data.csv", index=False)

In [10]:
df

,Price,Area,Position,Type,Benefit
0,"22,7 tỷ",140 m²,"Dự án The Crown - Vinhomes Ocean Park 3, Xã Ng...","Bán-Hưng Yên-Văn Giang-Nhà biệt thự, liền kề t...","Mức giá: 22,7 tỷ - Diện tích: 140 m² - Số tầng..."
2,"4,3 tỷ",75 m²,"Dự án The Greenery Hải Phòng, Phường An Lư, Th...","Bán-Hải Phòng-Thủy Nguyên-Shophouse, nhà phố t...","Mức giá: 4,3 tỷ - Diện tích: 75 m² - Hướng nhà..."
5,18 tỷ,75 m²,"Vinhomes Cổ Loa, Đường Cổ Loa, Xã Đông Hội, Đô...","Bán-Hà Nội-Đông Anh-Nhà biệt thự, liền kề tại ...",Mức giá: 18 tỷ - Diện tích: 75 m² - Pháp lý: S...
9,"18,4 tỷ","62,5 m²","Vinhomes Cổ Loa, Đường Cổ Loa, Xã Đông Hội, Đô...","Bán-Hà Nội-Đông Anh-Nhà biệt thự, liền kề tại ...","Mức giá: 18,4 tỷ - Diện tích: 62,5 m² - Hướng ..."
14,"2,6 tỷ",59 m²,"Dự án Vinhomes Ocean Park Gia Lâm, Xã Dương Xá...",Bán-Hà Nội-Gia Lâm-Căn hộ chung cư tại Vinhome...,"Mức giá: 2,6 tỷ - Diện tích: 59 m² - Số phòng ..."
...,...,...,...,...,...
71630,9 triệu/tháng,44 m²,"Centana Thủ Thiêm, Đường Mai Chí Thọ, Phường A...",Cho thuê-Hồ Chí Minh-Quận 2-Căn hộ chung cư tạ...,Mức giá: 9 triệu/tháng - Diện tích: 44 m² - Số...
72009,45 triệu/tháng,200 m²,"Đường Nguyễn Phước Lan, Phường Hòa Xuân, Cẩm L...",Cho thuê-Đà Nẵng-Cẩm Lệ-Nhà mặt phố tại đường ...,Mức giá: 45 triệu/tháng - Diện tích: 200 m² - ...
72389,38 triệu/tháng,200 m²,"266 Đường Bùi Minh Trực, Phường 6, Quận 8, Hồ ...",Cho thuê-Hồ Chí Minh-Quận 8-Nhà riêng tại đườn...,Mức giá: 38 triệu/tháng - Diện tích: 200 m² - ...
73535,"22,25 tỷ",80 m²,"Dự án Vinhomes Cổ Loa, Đường Cổ Loa, Xã Đông H...","Bán-Hà Nội-Đông Anh-Nhà biệt thự, liền kề tại ...","Mức giá: 22,25 tỷ - Diện tích: 80 m² - Số phòn..."


# Lưu vào trong elasticsearch 1 cột summary lại các nội dung của dữ liệu. Khi đó truy vấn vào vector này nhưng ta có thể trả về toàn bộ thông tin khác 

In [11]:
embed_model = SentenceTransformer('all-mpnet-base-v2')

In [12]:

df["SummaryVector"] = df.apply(lambda row: f"Giá nhà là: {row['Price']} có diện tích: {row['Area']}. \
    ở tại vị trí: {row['Position']} là 1 ngôi nhà dạng: {row['Type']} có những tiện ích như: {row['Benefit']}.", axis=1)

df["SummaryVector"] = df["SummaryVector"].apply(lambda x: embed_model.encode(x))

# Xóa document nếu muốn làm lại từ đầu 

In [13]:
index_name = 'rag'
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
    print(f"Index '{index_name}' deleted.")
es.indices.create(index=index_name, mappings=indexMapping)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'rag'})

# Lưu các index và Elasticsearch document

In [14]:
record_list = df.to_dict("records")
for record in record_list:
    try:
        es.index(index="rag", document=record)
    except Exception as e:
        print(e)


# Lấy các giá trị dựa vào query vector

In [15]:
max_candi = es.count(index="rag")["count"]
if max_candi > 0:
    query = {
    "field" : "SummaryVector",
    "query_vector" : embed_model.encode("Giá nhà tại Long Biên"),
    "k" : 5,
    "num_candidates" : max_candi , 
}
res = es.knn_search(index="rag", knn=query , source=["Price","Area","Position","Type","Benefit"])
results = []

for hit in res['hits']['hits']:
    print(hit['_source'])

{'Price': '14,2 tỷ', 'Area': '75 m²', 'Position': 'Phường Giang Biên, Long Biên, Hà Nội', 'Type': 'Bán-Hà Nội-Long Biên-Nhà riêng tại phường Giang Biên', 'Benefit': 'Mức giá: 14,2 tỷ - Diện tích: 75 m²'}
{'Price': '13 triệu/tháng', 'Area': '108 m²', 'Position': 'Dự án Sunshine Green Iconic, Phường Phúc Đồng, Long Biên, Hà Nội', 'Type': 'Cho thuê-Hà Nội-Long Biên-Căn hộ chung cư tại Sunshine Green Iconic', 'Benefit': 'Mức giá: 13 triệu/tháng - Diện tích: 108 m² - Số phòng ngủ: 3 phòng - Nội thất: Phòng ngủ full sàn gồ, Lô gia nối thông siêu rộng. * Điều hòa âm trần Samsung, thiết bị điện thông minh điều khiển từ xa. * Khóa cửa điện tử, thiết bị vệ sinh Grohe, tủ bếp cao cấp thương hiệu Hafele,'}
{'Price': '10 triệu/tháng', 'Area': '74 m²', 'Position': 'Sunshine Green Iconic, Phường Phúc Đồng, Long Biên, Hà Nội', 'Type': 'Cho thuê-Hà Nội-Long Biên-Căn hộ chung cư tại Sunshine Green Iconic', 'Benefit': 'Mức giá: 10 triệu/tháng - Diện tích: 74 m² - Số phòng ngủ: 2 phòng - Số phòng tắm, vệ 

C:\Users\Song\AppData\Local\Temp\ipykernel_19728\767653832.py:9: GeneralAvailabilityWarning: This API is in technical preview and may be changed or removed in a future release. Elastic will work to fix any issues, but features in technical preview are not subject to the support SLA of official GA features.
  res = es.knn_search(index="rag", knn=query , source=["Price","Area","Position","Type","Benefit"])
C:\Users\Song\AppData\Local\Temp\ipykernel_19728\767653832.py:9: ElasticsearchWarning: The kNN search API has been replaced by the `knn` option in the search API.
  res = es.knn_search(index="rag", knn=query , source=["Price","Area","Position","Type","Benefit"])
